In [1]:
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import labeled dataset
l_df = pd.read_csv('/content/drive/MyDrive/120321-V12/tweetid_userid_keyword_sentiments_emotions_Argentina.zip')

In [ ]:
l_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68645 entries, 0 to 68644
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tweet_ID             68645 non-null  int64  
 1   user_id              68645 non-null  int64  
 2   keywords             68645 non-null  object 
 3   tweet_timestamp      68645 non-null  object 
 4   valence_intensity    68645 non-null  float64
 5   fear_intensity       68645 non-null  float64
 6   anger_intensity      68645 non-null  float64
 7   happiness_intensity  68645 non-null  float64
 8   sadness_intensity    68645 non-null  float64
 9   sentiment            68645 non-null  int64  
 10  emotion              55983 non-null  object 
dtypes: float64(5), int64(3), object(3)
memory usage: 5.8+ MB


In [ ]:
l_df.head()

,tweet_ID,user_id,keywords,tweet_timestamp,valence_intensity,fear_intensity,anger_intensity,happiness_intensity,sadness_intensity,sentiment,emotion
0,1221975184682442753,3157061045,wuhan,2020-01-28 01:56:01,0.374,0.576,0.479,0.261,0.468,-1,F
1,1222091562580107270,55560080,wuhan,2020-01-28 09:38:28,0.401,0.651,0.457,0.270,0.490,-1,F
2,1222112877819179008,146560769,wuhan,2020-01-28 11:03:10,0.509,0.519,0.419,0.286,0.430,0,NaN
3,1222114892758446080,1161335189135486977,wuhan,2020-01-28 11:11:10,0.593,0.406,0.339,0.354,0.322,1,H
4,1222119134697271296,1217622758735085569,wuhan,2020-01-28 11:28:02,0.260,0.468,0.744,0.112,0.586,-2,A


In [ ]:
l_df.keywords.value_counts()

NameError: ignored

In [ ]:
l_df = l_df[l_df.keywords.isin(['covid','corona'])][['tweet_ID','sentiment']].reset_index(drop=True)

NameError: ignored

In [3]:
# Import extracted tweets using snscraper
df = pd.read_csv('/content/drive/MyDrive/tweets_extracted_IDs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937466 entries, 0 to 937465
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   Tweet Id  937466 non-null  int64
dtypes: int64(1)
memory usage: 7.2 MB


In [4]:
df.head()

,Tweet Id
0,1215249537834442753
1,1215247865066524675
2,1215076140412366848
3,1217191264858206209
4,1217063853894307842


In [ ]:
pd.merge(df,l_df, left_on='Tweet Id', right_on='tweet_ID')[['Tweet Id', 'sentiment']]

,Tweet Id,tweet_ID,sentiment
0,1254120166226608129,1254120166226608129,-1
1,1261419115404648448,1261419115404648448,-1
2,1263179893849886720,1263179893849886720,1
3,1264673527355060226,1264673527355060226,-1
4,1267600145434185728,1267600145434185728,-1
...,...,...,...
119,1399367177963180038,1399367177963180038,-1
120,1401999960585953284,1401999960585953284,1
121,1402319478533337092,1402319478533337092,0
122,1404531479623962626,1404531479623962626,1


### Labeling

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/tweets_extracted_IDs.csv')
for file in os.listdir('/content/drive/MyDrive/120321-V12'):
  print('----------------------------------------------------------------------------------------------------')
  country = file.replace('tweetid_userid_keyword_sentiments_emotions_','').replace('.zip','')
  print('Extracting sentiment labels from ',country)
  l_df = pd.read_csv('/content/drive/MyDrive/120321-V12/'+file, iterator=True, chunksize=2000000)
  for i, chunk in enumerate(l_df):
    print('Number of tweets : ', len(chunk))
    chunk_df = chunk[chunk.keywords.isin(['covid','corona'])][['tweet_ID','sentiment']].reset_index(drop=True)
    print('Number of tweets with reference to covid or corona : ',len(chunk_df))
    labeled_df = pd.merge(df,chunk_df, left_on='Tweet Id', right_on='tweet_ID')[['Tweet Id', 'sentiment']]
    print('Matches found : ',len(labeled_df))
    filename = '/content/drive/MyDrive/labeled_tweets/{0}_{1}.csv'.format(country,i)
    labeled_df.to_csv(filename, index=False)
    print('Saved tweets to ',filename)
  print('----------------------------------------------------------------------------------------------------')

----------------------------------------------------------------------------------------------------
Extracting sentiment labels from  Argentina
Number of tweets :  68645
Number of tweets with reference to covid or corona :  67463
Matches found :  124
Saved tweets to  /content/drive/MyDrive/labeled_tweets/Argentina_0.csv
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Extracting sentiment labels from  Australia
Number of tweets :  2000000
Number of tweets with reference to covid or corona :  1936445
Matches found :  3177
Saved tweets to  /content/drive/MyDrive/labeled_tweets/Australia_0.csv
Number of tweets :  2000000
Number of tweets with reference to covid or corona :  1980032
Matches found :  8701
Saved tweets to  /content/drive/MyDrive/labeled_tweets/Australia_1.csv
Number of tweets :  533585
Number of tweets with reference to cov

In [ ]:
path = '/content/drive/MyDrive/labeled_tweets/'
df = pd.DataFrame()
for file in os.listdir(path):
  country = file[:file.find('_')]
  temp = pd.read_csv(path+file)
  temp['country'] = country
  df = df.append(temp)
  print('{0} tweets added, size now : {1}'.format(len(temp),len(df)))
df.info()

124 tweets added, size now : 124
3177 tweets added, size now : 3301
8701 tweets added, size now : 12002
4070 tweets added, size now : 16072
2455 tweets added, size now : 18527
5344 tweets added, size now : 23871
7965 tweets added, size now : 31836
8782 tweets added, size now : 40618
7686 tweets added, size now : 48304
508 tweets added, size now : 48812
204 tweets added, size now : 49016
870 tweets added, size now : 49886
199 tweets added, size now : 50085
68 tweets added, size now : 50153
790 tweets added, size now : 50943
1101 tweets added, size now : 52044
3394 tweets added, size now : 55438
3080 tweets added, size now : 58518
296 tweets added, size now : 58814
170 tweets added, size now : 58984
6710 tweets added, size now : 65694
459 tweets added, size now : 66153
472 tweets added, size now : 66625
1050 tweets added, size now : 67675
363 tweets added, size now : 68038
2860 tweets added, size now : 70898
667 tweets added, size now : 71565
1891 tweets added, size now : 73456
412 tweet

In [ ]:
df.head()

,Tweet Id,sentiment,country
0,1254120166226608129,-1,Argentina
1,1261419115404648448,-1,Argentina
2,1263179893849886720,1,Argentina
3,1264673527355060226,-1,Argentina
4,1267600145434185728,-1,Argentina


In [ ]:
df.country.value_counts()

United States     225874
United Kingdom     56799
Canada             32232
Australia          15948
India               8729
Ireland             6710
New Zealand         4800
South Africa        4106
Nigeria             3527
Pakistan            1891
Kenya               1050
France               870
Philippines          616
Brazil               508
Japan                472
Italy                459
Singapore            412
Malaysia             363
Spain                362
China                204
Denmark              199
Indonesia            170
Argentina            124
South Korea          119
Colombia             107
Russia                94
Turkey                81
Thailand              72
Vietnam               32
Iran                  24
Name: country, dtype: int64

In [ ]:
df.to_csv('/content/drive/MyDrive/labeled_tweets.csv', index=False)

#### Merging with extracted data

In [5]:
df = pd.read_csv('/content/drive/MyDrive/labeled_tweets.csv')

In [6]:
dfs = [] # an empty list to store the data frames
path_to_json = '/content/drive/MyDrive/data_with_no_replies_links'
json_files = [path_to_json+'/'+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

dfs = [] # an empty list to store the data frames
for file in json_files:
    with open(file,'r') as file:
        data = pd.read_json(file, lines=True) # read data frame from json file
        dfs.append(data) # append the data frame to the list

temp = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937466 entries, 0 to 937465
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Datetime  937466 non-null  datetime64[ns]
 1   Tweet Id  937466 non-null  int64         
 2   Text      937466 non-null  object        
 3   Username  937466 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 28.6+ MB


In [7]:
temp.to_csv('/content/drive/MyDrive/data_with_no_replies_links.csv', index=False)

In [8]:
def process_sent_labels(x):
  if(x==0):
    return 'Neutral'
  elif(x<0):
    return 'Negative'
  elif(x>0):
    return 'Positive'

labeled_df = pd.merge(df, temp, on ='Tweet Id', how='right')
labeled_df['sentiment'] = labeled_df.sentiment.apply(lambda x:process_sent_labels(x))
labeled_df.sentiment.value_counts()

Negative    245823
Positive     76415
Neutral      46820
Name: sentiment, dtype: int64

In [9]:
labeled_df[labeled_df.sentiment.notnull()].head()

,Tweet Id,sentiment,country,Datetime,Text,Username
1,1530337446269001730,Positive,United States,2022-05-27 23:57:34,I’m ready to read these behind the scenes stor...,ShawnZGatlin
3,1530337329042235393,Negative,United States,2022-05-27 23:57:06,Covid permanently ruined my sense of smell &am...,ReverieLOVE
4,1530337273614581762,Negative,United States,2022-05-27 23:56:52,My three year old’s COVID has relapsed. I hate...,msruggerio
5,1530337250978017283,Positive,United States,2022-05-27 23:56:47,Delivering my husband his paxlovid within 20 m...,darakass
6,1530337166341328896,Negative,Australia,2022-05-27 23:56:27,Repeat ad nauseum for person after person:\n\n...,AugersMonster


In [10]:
labeled_df[labeled_df.sentiment.notnull()].Datetime.min(), labeled_df[labeled_df.sentiment.notnull()].Datetime.max()

(Timestamp('2020-01-28 10:23:10'), Timestamp('2022-06-01 23:59:07'))

In [11]:
labeled_df.to_csv('/content/drive/MyDrive/labeled_data_with_no_replies_links.csv', index=False)